In [87]:
# !pip install flair beautifulsoup4 pdfkit
# !sudo apt-get install wkhtmltopdf

[sudo] password for gabriel: 


In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from flair.data import Sentence 
from flair.models import SequenceTagger
from tqdm import tqdm
from segtok.segmenter import split_single

df = pd.read_csv('furniture stores pages.csv')

tagger = SequenceTagger.load('ner-ontonotes-large')  # ner-ontonotes-large

URL = "https://vastinterior.com.au/products/samson-daybed-single" 
page = requests.get(URL)

2022-01-23 21:41:48,291 --------------------------------------------------------------------------------
2022-01-23 21:41:48,291 The model key 'ner-ontonotes-large' now maps to 'https://huggingface.co/flair/ner-english-ontonotes-large' on the HuggingFace ModelHub
2022-01-23 21:41:48,291  - The most current version of the model is automatically downloaded from there.
2022-01-23 21:41:48,291 --------------------------------------------------------------------------------
2022-01-23 21:41:48,797 loading file /home/gabriel/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


In [20]:
soup = BeautifulSoup(page.content, "html.parser")
results = soup.prettify()
# results = soup.get_text()
filtered = [line for line in [re.sub('\n+', '\n', re.sub('[ \t]+', ' ',  line)) for line in results.splitlines()]
            if line != '' and line != ' ' and line != '\n' and line != '\t']

sentences = [Sentence(sent, use_tokenizer=True) for line in filtered for sent in split_single(line)
             if sent != '' and sent != ' ' and sent != '\n' and sent != '\t' and len(sent) <= 512]
# print(sentences)

def batch(iterable, n=16):
    l = len(iterable)
    for ndx in tqdm(range(0, l // 3, n)):
        yield iterable[ndx:min(ndx + n, l)]

for sent_batch in batch(sentences):        
    tagger.predict(sent_batch)

for sent in sentences:
    for entity in sent.get_spans('ner'):
        if entity.tag == 'PRODUCT' and entity.score > 0.9:
            print(entity)
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:35<00:00,  2.06it/s]

Span [3,4,5]: "Single Day Bed"   [− Labels: PRODUCT (0.9998)]
Span [4]: "iPhone"   [− Labels: PRODUCT (0.9995)]
Span [17]: "iPad"   [− Labels: PRODUCT (0.9999)]
Span [32,33,34,35,36]: "Samson Daybed Single - 230X83X90"   [− Labels: PRODUCT (0.9667)]
Span [41,42,43,44,45]: "Samson Daybed Single - 230X83X90"   [− Labels: PRODUCT (0.9683)]


In [ ]:
# def build_dataset(urls, char_limit=10000):

#     webpages = []

#     for url in urls:

#         try:

#             data = requests.get(url)
#             soup = BeautifulSoup(data.content, 'html.parser')
#             tags = soup.find_all()

#             text = ''
#             blacklist = [
#                 '[document]',
#                 'noscript',
#                 'header',
#                 'html',
#                 'meta',
#                 'head',
#                 'input',
#                 'script',
#                 'style',
#                 'title',
#                 'script'
#             ]

#             for t in tags:
#                 if t not in blacklist and t.parent.name not in blacklist:
#                     text += '{} '.format(t.text)

#             clean_text = " ".join(text.split())
#             if len(clean_text) > char_limit:
#                 clean_text = clean_text[:char_limit]

#             webpages.append(clean_text)

#         except Exception as e:
#             logging.exception(str(e))
#             webpages.append('')

#     df = pd.DataFrame(webpages, columns=['raw_text'])
#     # df = df.replace(np.nan, '', regex=True)
#     return df